# Retrieval Augmented Generation (RAG) with OpenAI and Qdrant

In the ever-evolving landscape of AI, the consistency and reliability of Large Language Models (LLMs) remain a challenge. While these models can understand statistical relationships between words, they often fail to provide accurate factual responses. Because their internal knowledge may not be accurate, outputs can range from spot-on to nonsensical. Retrieval Augmented Generation (RAG) is a framework designed to bolster the accuracy of LLMs by grounding them in external knowledge bases. In this example, we'll demonstrate a streamlined  implementation of the RAG pipeline using only Qdrant and OpenAI SDKs. By harnessing Flag embedding's power, we can bypass additional frameworks' overhead. 
    
This example assumes you understand the architecture necessary to carry out RAG. If this is new to you, please look at some introductory readings:
* [Retrieval-Augmented Generation: To add knowledge](https://eugeneyan.com/writing/llm-patterns/#retrieval-augmented-generation-to-add-knowledge)
* [On improving LLMs quality](https://deploy-preview-328--condescending-goldwasser-91acf0.netlify.app/articles/improving-llms-quality/)

## Prerequisites

Let's start setting up all the pieces to implement the RAG pipeline. We will only use Qdrant and OpenAI SDKs, without any third-party libraries. 

### Preparing the environment

We need just a few dependencies to implement the whole application, so let's start with installing the dependencies.

In [1]:
!pip install qdrant-client fastembed openai

[Qdrant](https://qdrant.tech) will act as a knowledge base providing the context information for the prompts we'll be sending to the LLM. There are various ways of running Qdrant, but we'll simply use the Docker container.

In [2]:
!docker run -p "6333:6333" -p "6334:6334" --name "rag-openai-qdrant" --rm -d qdrant/qdrant:latest

066db7a31c0261cbbb221eec95b9ce214ae306b6601afbdb03cefa068a9cf9ed


### Creating the collection

Qdrant [collection](https://qdrant.tech/documentation/concepts/collections/) is the basic unit of organizing your data. Each collection is a named set of points (vectors with a payload) among which you can search. After connecting to our running Qdrant container, we can check whether we already have some collections.

In [3]:
import qdrant_client

client = qdrant_client.QdrantClient("http://localhost:6333", prefer_grpc=True)
client.get_collections()

CollectionsResponse(collections=[])

### Building the knowledge base

Qdrant will use vector embeddings of our facts to enrich the original prompt with some context. Thus, we need to store the vector embeddings and the texts used to generate them. All our facts will have a JSON payload with a single attribute and look as follows:

```json
{
    "document": "Binary Quantization is a method of reducing the memory usage even up to 40 times!"
}
```

This structure is required by [FastEmbed](https://qdrant.github.io/fastembed/), a library that simplifies managing the vectors, as you don't have to calculate them on your own. It's also possible to use an existing collection, However, all the code snippets will assume this data structure. Adjust your examples to work with a different schema.

FastEmbed will automatically create the collection if it doesn't exist. Knowing that we are set to add our documents to a collection, which we'll call `knowledge-base`.

In [4]:
client.add(
    collection_name="knowledge-base",
    documents=[
        "Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!",
        "Docker helps developers build, share, and run applications anywhere — without tedious environment configuration or management.",
        "PyTorch is a machine learning framework based on the Torch library, used for applications such as computer vision and natural language processing.",
        "MySQL is an open-source relational database management system (RDBMS). A relational database organizes data into one or more data tables in which data may be related to each other; these relations help structure the data. SQL is a language that programmers use to create, modify and extract data from the relational database, as well as control user access to the database.",
        "NGINX is a free, open-source, high-performance HTTP server and reverse proxy, as well as an IMAP/POP3 proxy server. NGINX is known for its high performance, stability, rich feature set, simple configuration, and low resource consumption.",
        "FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Python type hints.",
        "SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. You can use this framework to compute sentence / text embeddings for more than 100 languages. These embeddings can then be compared e.g. with cosine-similarity to find sentences with a similar meaning. This can be useful for semantic textual similar, semantic search, or paraphrase mining.",
        "The cron command-line utility is a job scheduler on Unix-like operating systems. Users who set up and maintain software environments use cron to schedule jobs (commands or shell scripts), also known as cron jobs, to run periodically at fixed times, dates, or intervals.",
    ]
)

['bd4a7b79a81947f2a5e94b896a468ce1',
 '8611b165664b4f908710cac0de0330a8',
 'b77c4297ea2d4d23a29a5a79feceb250',
 '26c07c7971444365b4486e4a6925aeed',
 '539944a8711047b19642c8238231d64a',
 '3ed7cebd4b354f54b50ee817eebb23e9',
 'fb2d523bd50c4563a7d36de3cd2962ca',
 'ab94cc63ba53460fbad003a5e2b69e43']

## Retrieval Augmented Generation

RAG changes the way we interact with Large Language Models. We're converting a knowledge-oriented task, in which the model may create a counterfactual answer, into a language-oriented task. The latter expects the model to extract meaningful information and generate an answer. LLMs, when implemented correctly, are supposed to be carrying out language-oriented tasks.

The task starts with the original prompt sent by the user. The same prompt is then vectorized and used as a search query for the most relevant facts. Those facts are combined with the original prompt to build a longer prompt containing more information.

But let's start simply by asking our question directly.

In [5]:
prompt = """
What tools should I need to use to build a web service using vector embeddings for search?
"""

Using OpenAI API requires providing the API key. Our example demonstrates setting the `OPENAI_API_KEY` using an environmental variable.

In [6]:
import os

# Fill the environmental variable with your own OpenAI API key
# See: https://platform.openai.com/account/api-keys
os.environ["OPENAI_API_KEY"] = "<< PASS YOUR OWN KEY >>"

Now we can finally call the completion service.

In [7]:
import openai

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", 
    messages=[
        {"role": "user", "content": prompt},
    ]
)
print(completion["choices"][0]["message"]["content"])

To build a web service using vector embeddings for search, you would need several tools. Here are some essential ones:

1. Programming Language: Depending on your preference and requirements, you can choose a programming language like Python, Java, or Node.js for building the web service.

2. Web Framework: A web framework helps in developing web applications efficiently. Popular choices include Flask (Python) or Spring Boot (Java).

3. Embedding Models: You would need vector embedding models to represent your data for search. Some popular models include Word2Vec, GloVe, or BERT, depending on your application domain and requirements.

4. Vector Embedding Libraries: To work with vector embeddings effectively, you may need libraries like TensorFlow, Gensim, or PyTorch to load and manipulate the embeddings.

5. Database: You would require a database to store and retrieve the information you want to search. Common choices include MySQL, PostgreSQL, or MongoDB.

6. Search Engine: You need a

### Extending the prompt

Even though the original answer sounds credible, it didn't answer our question correctly. Instead, it gave us a generic description of an application stack. To improve the results, enriching the original prompt with the descriptions of the tools available seems like one of the possibilities. Let's use a semantic knowledge base to augment the prompt with the descriptions of different technologies!

In [8]:
results = client.query(
    collection_name="knowledge-base",
    query_text=prompt,
    limit=3,
)
results

[QueryResponse(id='bd4a7b79-a819-47f2-a5e9-4b896a468ce1', embedding=None, metadata={'document': 'Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!'}, document='Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!', score=0.795905590057373),
 QueryResponse(id='3ed7cebd-4b35-4f54-b50e-e817eebb23e9', embedding=None, metadata={'document': 'FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Python type hints.'}, document='FastAPI is a moder

We used the original prompt to perform a semantic search over the set of tool descriptions. Now we can use these descriptions to augment the prompt and create more context.

In [9]:
context = "\n".join(r.document for r in results)
context

'Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!\nFastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Python type hints.\nSentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. You can use this framework to compute sentence / text embeddings for more than 100 languages. These embeddings can then be compared e.g. with cosine-similarity to find sentences with a similar meaning. This can be useful for semantic textual similar, semantic search, or paraphrase mining.'

Finally, let's build a metaprompt, the combination of the assumed role of the LLM, the original question, and the results from our semantic search that will force our LLM to use the provided context. 

By doing this, we effectively convert the knowledge-oriented task into a language task and hopefully reduce the chances of hallucinations. It also should make the response sound more relevant.

In [10]:
metaprompt = f"""
You are a software architect. 
Answer the following question using the provided context. 
If you can't find the answer, do not pretend you know it, but answer "I don't know".

Question: {prompt.strip()}

Context: 
{context.strip()}

Answer:
"""

# Look at the full metaprompt
print(metaprompt)


You are a software architect. 
Answer the following question using the provided context. 
If you can't find the answer, do not pretend you know it, but answer "I don't know".

Question: What tools should I need to use to build a web service using vector embeddings for search?

Context: 
Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!
FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Python type hints.
SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. You can use this framework to compute sentence / text embeddings for more than 100 languages. These embeddings can then be compared e.g. with cosine-similarity to find sentences

Our current prompt is much longer, and we also used a couple of strategies to make the responses even better:

1. The LLM has the role of software architect.
2. We provide more context to answer the question.
3. If the context contains no meaningful information, the model shouldn't make up an answer.

Let's find out if that works as expected.

In [11]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": metaprompt},
    ],
    timeout=10.0,
)
print(completion["choices"][0]["message"]["content"])

To build a web service using vector embeddings for search, you would need to use Qdrant as the vector database and vector similarity search engine. Additionally, you can use FastAPI as the web framework for building the API. To compute the vector embeddings, you can utilize SentenceTransformers, which is a Python framework for generating sentence/text embeddings. These tools would enable you to create a web service for matching, searching, recommending, and more based on vector embeddings.


### Testing out the RAG pipeline

By leveraging the semantic context we provided our model is doing a better job answering the question. Let's enclose the RAG as a function, so we can call it more easily for different prompts.

In [12]:
def rag(question: str, n_points: int = 3) -> str:
    results = client.query(
        collection_name="knowledge-base",
        query_text=question,
        limit=n_points,
    )

    context = "\n".join(r.document for r in results)

    metaprompt = f"""
    You are a software architect. 
    Answer the following question using the provided context. 
    If you can't find the answer, do not pretend you know it, but answer "I don't know".
    
    Question: {question.strip()}
    
    Context: 
    {context.strip()}
    
    Answer:
    """

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": metaprompt},
        ],
        timeout=10.0,
    )
    return completion["choices"][0]["message"]["content"]

Now it's easier to ask a broad range of questions.

In [13]:
rag("What can the stack for a web api look like?")

"The stack for a web API can vary depending on the specific requirements and technologies used, but a common stack for a web API could include the following components:\n\n1. FastAPI: As mentioned in the context, FastAPI is a modern web framework for building APIs with Python.\n\n2. NGINX: NGINX can be used as a reverse proxy server to handle incoming requests and distribute them to the appropriate FastAPI instances. NGINX is known for its high performance and can help improve the scalability and reliability of the API.\n\n3. Docker: Docker can be utilized to containerize the FastAPI application along with any necessary dependencies. This allows for easy deployment and scalability, as Docker containers can be easily replicated and managed.\n\nIn addition to these components, the stack may also include other elements such as a database (e.g. PostgreSQL, MySQL) for storing and retrieving data, a cache (e.g. Redis, Memcached) for improving performance, and a message broker (e.g. RabbitMQ,

In [14]:
rag("Where is the nearest grocery store?")

"I don't know."

Our model can now:

1. Take advantage of the knowledge in our vector datastore.
2. Answer, based on the provided context, that it can not provide an answer.

We have just shown a useful mechanism to mitigate the risks of hallucinations in Large Language Models.

### Cleaning up the environment

If you wish to continue playing with the RAG application we created, don't do the code below. However, it's always good to clean up the environment, so nothing is left dangling. We'll show you how to remove the Qdrant container.

In [15]:
!docker kill rag-openai-qdrant
!docker rm rag-openai-qdrant